# Sex-related differences in the human microbiome using curatedMetagenomicData 3 and the python 3 language

Paolo Manghi, (paolo.manghi@unitn.it)

This notebook contains the instructions to run a meta-analysis of sex-related contrasts in the human gut microbiome, using `curatedMetagenomicDataCLI` and a set of freely-available python programs. If not running in docker, see the [installation instructions](installation.ipynb).

As described here, we are now going to

1. create a folder called `species_abundances_from_cMD3CLI`
2. go in that directory
3. download all the taxonomic profiles using `curatedMetagenomicDataCLI`

The step 3 may take some time.

In [ ]:
%%bash

mkdir $HOME/species_abundances_from_cMD3CLI
cd $HOME/species_abundances_from_cMD3CLI
curatedMetagenomicData -m "*relative_abundance"
cd $HOME

The flag "-m" will attach the per-sample metadata available in curatedMetagenomicData 3 to their taxonomic profiles.
We now switch to a python 3 set of instructions that can be used to perform the main analysis of Figure 2, panel a, of the paper "***". 

The following instructions are meant as an example and also to clarify some aspect of the inner-code: the whole analysis is avaiable, as explained, via command-line programs at [https://github.com/waldronlab/curatedMetagenomicAnalyses/tree/main/python_tools] and can be all completed from the shell.

## Imports section

So we must first import the basic modules to work with cMDCLI in a meta-analytical way. Note: If your `python_modules` and `python_tools` directories are in a different location, correct the paths below.

In [11]:
import os
import sys

sys.path.append(os.path.join(os.environ["HOME"], "curatedMetagenomicAnalyses/python_modules"))
sys.path.append(os.path.join(os.environ["HOME"], "curatedMetagenomicAnalyses/python_tools"))

import pandas as pd
import numpy as np

## The following two modules handle either 
## a meta-analysis based on a std. linear model
## OR a std. differential abundance analysis.
## This is useful as some biological questions are
## commonly explorable just in one dataset.
from meta_analysis_runner import meta_analysis_with_linear_model
from meta_analysis_runner import analysis_with_linear_model

## Construction of the dataset

Run the following code to see its help page.

In [ ]:
%%bash

python3 $HOME/curatedMetagenomicAnalyses/python_tools/meta_analysis_data.py -h

In [13]:
## The function `select` builds a dataset according to the requirements/constraints passed by the user. 
## Now, the `select` function can be imported as a module and used:
## To to this, it is enough to pass it a dictionary of the parameters,
## as follows:

from meta_analysis_data import select

## Following from the "Help" page, we next try to build 
## a dataset to study the Sex-contrast in the healthy, human, gut microbiome:

params = {
    'input_folder': os.path.join(os.environ["HOME"], "species_abundances_from_cMD3CLI"),
    "output_dataset": os.path.join(os.environ["HOME"], "a_dataset_for_the_sex_contrast_in_gut_species.tsv"),
    "min": ["age:16"],
    "max": [],
    "cat": ["study_condition:control", "body_site:stool"], 
    "multiple": -1,
    "min_perc": ["gender:25"],
    "cfd":["BMI"], 
    "iqr": [],
    "minmin": "gender:40",
    "study_identifier": "study_name", 
    "verbose": False, 
    "debug": False,
    "binary": [],
    "search": [],
    "exclude": []
}

These are the parameters that are needed to narrow this dataset. Normally, these are inserted via command-line, 
but python is flexible on this.

Now we write a dataset table: a dataset-table is meant here as a table with Samples as columns-ID, and metadata + features as index (row-names). Features are then distingushed from metadata based on the "feat-id" keyword.

In [ ]:
select(params)

Now, you should have saved a file named `a_dataset_for_the_sex_contrast_in_gut_species.tsv`. 

At June, 2021, this dataset should store 4007 sample-IDs (columns) + 1 column index.
The current dataset is not yet usable to perform a meta-analysis as the compositional 
data returned by MetaPhlAn3 software are not recommended for work with most statistical methods.

Among the transformation more often applied, there are the *Centered-Log-Ratio* (CLR) and the *arcsin-square-root* of 
the relative-abundance-corresponding proportions. This last method is widely applied in meta-analyses on 
microbiome-related questions as it considers equally all the zeros independent from the dataset.

We thus will apply the arcsin-square root, but a utility script is currently available to apply both. Run the following code to read its help page.

In [ ]:
%%bash

python3 $HOME/curatedMetagenomicAnalyses/python_tools/apply_a_transform.py -h

In [ ]:
## The general procedure to transform only the **features**
## in a dataset-merged-table is quite simple.
## We must first read the table we have created:
sex_contrast_dataset = pd.read_csv(os.path.join(os.environ["HOME"], "a_dataset_for_the_sex_contrast_in_gut_species.tsv"), sep="\t", \
        header=0, index_col=0, low_memory=False, engine="c").fillna("NA")

## We then identify it's features
feats = [j for j in sex_contrast_dataset.index.tolist() if ("s__" in j)]

## We transform the data
for sample in sex_contrast_dataset.columns.tolist():
    sex_contrast_dataset.loc[feats, sample] = sex_contrast_dataset.loc[feats, sample].values.astype(float)
    sex_contrast_dataset.loc[feats, sample] /= np.sum(sex_contrast_dataset.loc[feats, sample].values)
    sex_contrast_dataset.loc[feats, sample] = np.arcsin(np.sqrt(\
        sex_contrast_dataset.loc[feats, sample].values.astype(float)))
    
## Now that we have transformed the data we can save a table which is suitable for the analyses:
sex_contrast_dataset.index.name = "sample_id"
sex_contrast_dataset.to_csv(os.path.join(os.environ["HOME"], "a_dataset_for_the_sex_contrast_in_gut_species_arcsin.tsv"), sep="\t", \
        header=True, index=True)

It's now time to use the dataset we have created to perform a meta-analysis.
The meta-analysis we'll perform is a standard meta-analysis (e.g. a weighted-average of several 
effect-sizes, computed over several, independent populations).

The effect-size will be in the class of the differences of means (Cohen's d) and will characterize 
the difference between males and females (men and women).

The only difference, with respect to a canonical paradigm of meta-analysis, is that we will extract, 
each time, **the Cohen'd effect size from the sex-relative coefficient of an ordinary least squares (OLS)
model**. Beside being averaged over 15 populations, these effect-sizes will be so adjusted by age and by BMI
in addition.

Now, the program allowing you to run directly the meta-analysis is called `metaanalyze.py`, and 
asks for a short set of parameters in order to understand how to perform the desired meta-analysis.

We will now run, **manually**, the basic steps of this program, in order to show them explicitly:

In [17]:
## First, we reproduce part of the program's import section
from meta_analyses import paule_mandel_tau
from meta_analyses import RE_meta_binary
from meta_analyses import RE_meta

## we specify the important parameters
outfile = "sex_4007_individuals_meta_analysis"
heterogeneity = "PM" ## this corresponds to the default of the program
study_id = "study_name" ## this is needed in order to segregate the populations
type_of_analysis = "CLS" ## CLASSIFY (SEE AFTER)

We also need to specifiy a `formula` for the **model** that will be optimized before to compute the effect-size.
The model will be applied to all the features in `feats`. Given this, we don't have to specify the Y, but only the **X(s)**.

The first X will be the main predictor, and the effect-sizes will be computed on that.
The following X will be used to adjuste the model.

NOTE THAT:
1) Though this is not needed for the main one, **categorical** covariates must be explicitly indicated via **C(name of c.)**.

2) Though the first variable is automatically detected as categorical (if `type_of_analysis == "CLS"`), we must
    specify the **positive** and the **negative** direction of the analysis we are going to perform.

In [18]:
formula = "gender + age + BMI"
control_side, case_side = "female", "male"

We now can run the meta-analysis main module, `metaanalyze.py`.
The module can be run from command-line with a minimum parameters, provided that a suitable dataset has 
been built.

Run the following to view the help page for `metaanalyze.py`.

In [ ]:
%%bash

python3 $HOME/curatedMetagenomicAnalyses/python_tools/metaanalyze.py -h

In [20]:
ma = meta_analysis_with_linear_model(\
    sex_contrast_dataset, 
    formula, \
    study_id, \
    feats, \
    outfile + ".tsv", \
    type_of_analysis, \
    heterogeneity, \
    pos=case_side, \
    neg=control_side
    )

In [ ]:
ma.random_effect_regression_model()

Now that we have computed a meta-analysis, we want to plot it. Meta-analyses and Metagenomics are both dealing
each with a problem of multidimensionality of the results, so coupling them requires to do our best to fit the 
different layers of results in one figure.

In particular, we can plot many forest-plots in a single figure with the script: `draw_figure_with_ma.py`.
Run the following to see its help page.

In [ ]:
%%bash

python3 $HOME/curatedMetagenomicAnalyses/python_tools/draw_figure_with_ma.py -h

In order to call the program from python, we have to prepare a dictionary, faking its command-line arguments.
Also, the program is able to plot several meta-analysis in the same set of axes, though we will for now 
limit ourselves to the **basic usage**.

In [ ]:
figure_params = {
    "metaanalysis": [os.path.join(os.environ["HOME"], "sex_4007_individuals_meta_analysis.tsv")],
    "narrowed": True,
    "boxes": True,
    "relative_abundances": [os.path.join(os.environ["HOME"], "a_dataset_for_the_sex_contrast_in_gut_species.tsv")],
    "imp": 30, 
    "how": "first",
    "positive_direction": "sex: male",
    "negative_direction": "sex: female",
    "x_axis": "standardized mean difference",
    "y_axis": "",
    "title": "meta-analysis of sex-related microbial species",
    "e_suff": ["_Effect"],
    "q_suff": ["_Qvalue"],
    "prevalence": [""],
    "min_prevalence": [0.01],
    "min_ab": [0.000],
    "min_studies": [4],
    "markers": False,
    "outfile": None,
    "random_effect": ["RE_Effect"],
    "confint": ["RE_conf_int"],
    "random_effect_q": ["RE_Effect_Qvalue"],
    "color_red": ["goldenrod"],
    "color_blue": ["dodgerblue"],
    "color_black": ["black"],
    "diam_marker": ["D"],
    "important_lines": [0.20],
    "a_single": 0.2,
    "a_random": 0.05, 
    "dotsize": 9,
    "diamsize": 17,
    "neg_max_rho": 0.8,
    "pos_max_rho": 0.8,
    "legloc": "best"
}

## We then import the main function
from draw_figure_with_ma import draw_figure

## We run it
draw_figure(figure_params, show=True)